PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/26 14:27:42 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/26 14:27:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 14:27:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 보스턴 주택 가격 예측 모델 만들기




In [3]:
spark

In [4]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2024-06-26 13:02:48--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 3.5.80.208, 3.5.86.174, 3.5.76.13, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|3.5.80.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K   225KB/s    in 0.2s    

2024-06-26 13:02:49 (225 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [5]:
!ls -tl

total 688
-rw-r--r-- 1 sangwon sangwon  31965 Jun 26 12:15 PySpark_ML_실습1.ipynb
-rw-r--r-- 1 sangwon sangwon 195247 Jun 26 12:14 PySpark_Write_실습.ipynb
-rw-r--r-- 1 sangwon sangwon   1716 Jun 25 17:30 bucketDemo.py
-rw-r--r-- 1 sangwon sangwon  21777 Jun 25 17:24 PySpark_FileSystem_Partitioning.ipynb
drwxr-xr-x 3 sangwon sangwon   4096 Jun 25 17:22 spark-warehouse
drwxr-xr-x 5 sangwon sangwon   4096 Jun 25 17:22 metastore_db
-rw-r--r-- 1 sangwon sangwon    814 Jun 25 17:22 derby.log
-rw-r--r-- 1 sangwon sangwon  22795 Jun 25 16:41 PySpark_Schema_Evolution.ipynb
drwxr-xr-x 5 sangwon sangwon   4096 Jun 25 16:36 dataOutput
drwxr-xr-x 2 sangwon sangwon   4096 Jun 20 01:17 wordcount
drwxr-xr-x 4 sangwon sangwon   4096 Jun 20 01:17 join_broadcast_join
-rw-r--r-- 1 sangwon sangwon  29648 Jun 20 01:17 PySpark_ML_실습2.ipynb
-rw-r--r-- 1 sangwon sangwon  71580 Jan 26  2023 schema3.parquet
-rw-r--r-- 1 sangwon sangwon  49915 Jan 26  2023 schema1.parquet
-rw-r--r-- 1 sangwon sangwon  57

In [4]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [6]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [7]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [8]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [9]:
data_2 = assembler.transform(data)

In [10]:
data_2.show()

24/06/26 14:28:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [11]:
train, test = data_2.randomSplit([0.7, 0.3])

In [12]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

24/06/26 14:28:23 WARN Instrumentation: [6b5285f7] regParam is zero, which might cause numerical instability and overfitting.
24/06/26 14:28:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/26 14:28:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


## 모델 성능 측정

In [13]:
evaluation_summary = model.evaluate(test)

In [14]:
evaluation_summary

In [15]:
evaluation_summary.meanAbsoluteError

3.3130906259182953

In [16]:
evaluation_summary.rootMeanSquaredError

4.382023512576681

In [17]:
evaluation_summary.r2

0.7487619298289795

## 모델 예측값 살펴보기 

In [18]:
predictions = model.transform(test)

In [19]:
predictions.show()

+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.00906|90.0| 2.97|   0|   0.4|7.088|20.8| 7.3073|  1|285|   15.3|394.72| 7.85|32.2|[0.00906,90.0,2.9...|31.083504978653067|
|0.01096|55.0| 2.25|   0| 0.389|6.453|31.9| 7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|27.464475518419256|
|0.01311|90.0| 1.22|   0| 0.403|7.249|21.9| 8.6966|  5|226|   17.9|395.93| 4.81|35.4|[0.01311,90.0,1.2...|30.630856043471702|
| 0.0136|75.0|  4.0|   0|  0.41|5.888|47.6| 7.3197|  3|469|   21.1| 396.9| 14.8|18.9|[0.0136,75.0,4.0,...|15.007254744180884|
|0.01709|90.0| 2.02|   0|  0.41|6.728|36.1|12.1265|  5|187|   17.0|384.46|  4.5|30.1|[0.01709,90.0,2.0...|25.386043659

In [20]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|32.2|[0.00906,90.0,2.9...|31.083504978653067|
|22.0|[0.01096,55.0,2.2...|27.464475518419256|
|35.4|[0.01311,90.0,1.2...|30.630856043471702|
|18.9|[0.0136,75.0,4.0,...|15.007254744180884|
|30.1|[0.01709,90.0,2.0...|25.386043659441107|
|31.1|[0.02187,60.0,2.9...|31.938805828767663|
|16.5|[0.02498,0.0,1.89...|22.132361558703256|
|21.6|[0.02731,0.0,7.07...|25.392852500876373|
|26.6|[0.02899,40.0,1.2...| 21.39941448820728|
|28.7|[0.02985,0.0,2.18...|25.708982458458745|
|17.5|[0.03113,0.0,4.39...|16.498050184737185|
|19.5|[0.03427,0.0,5.19...|20.543031235991208|
|22.9|[0.03551,25.0,4.8...|25.406770406313612|
|35.4|[0.03705,20.0,3.3...|34.607989940831025|
|23.2|[0.03871,52.5,5.3...|27.394579783210226|
|28.0|[0.04113,25.0,4.8...|28.232552962026784|
|22.9|[0.04203,28.0,15....| 28.93356727384368|
|20.6|[0.04294,28.0,15....|27.331431070414677|
|18.2|[0.0430

In [21]:
model.save("boston_housing_model")

In [22]:
!ls boston_housing_model

data  metadata


In [23]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 sangwon sangwon 4096 Jun 26 14:30 data
drwxr-xr-x 2 sangwon sangwon 4096 Jun 26 14:30 metadata


In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [30]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model" 
model.save(path)

In [31]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [32]:
predictions2 = loaded_model.transform(test)

In [33]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|29.1|[0.01439,60.0,2.9...|31.863971610226315|
|24.5|[0.01501,80.0,2.0...|27.800912626874748|
|44.0|[0.01538,90.0,3.7...|38.081559279682246|
|23.9|[0.02543,55.0,3.7...|27.969747095640912|
|30.8|[0.02763,75.0,2.9...|31.575873666286917|
|25.0|[0.02875,28.0,15....|29.689655131070666|
|28.5|[0.03502,80.0,4.9...|33.855798290686124|
|45.4|[0.03578,20.0,3.3...|39.495929078055646|
|22.0|[0.03932,0.0,3.41...|27.678841182848963|
|33.3|[0.04011,80.0,1.5...|36.633177721713174|
|22.9|[0.04203,28.0,15....| 29.65081329317448|
|18.2|[0.04301,80.0,1.9...|14.312466963142263|
|20.5|[0.04337,21.0,5.6...| 24.52586127306892|
|19.8|[0.04544,0.0,3.24...|21.560592700808435|
|23.3|[0.0456,0.0,13.89...|27.974279873488676|
|17.1|[0.05023,35.0,6.0...|20.027398326196785|
|22.5|[0.05188,0.0,4.49...| 22.19486700208488|
|36.1|[0.05515,33.0,2.1...| 33.36611368642783|
|29.0|[0.0556